In [ ]:
!pip install tensorflow
!pip install opencv-python
!pip install spotipy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.5/261.5 kB 14.5 MB/s eta 0:00:00


In [ ]:
# Import necessary libraries
import tensorflow as tf
import numpy as np
import cv2
from google.colab import files
from PIL import Image
import spotipy
from spotipy.oauth2 import SpotifyOAuth

# Load pre-trained emotion model
uploaded_model = files.upload()  # Upload your pre-trained model file
# Load the model with the 'compile' argument set to False to avoid loading the optimizer
model = tf.keras.models.load_model("fer2013_mini_XCEPTION.102-0.66.hdf5", compile=False)
# Replace with your actual model file name

# Compile the model with the desired optimizer and learning rate
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), # Replace with your desired optimizer and learning rate
              loss='categorical_crossentropy', # Replace with your desired loss function
              metrics=['accuracy']) # Replace with your desired metrics

# Upload image instead of using webcam
uploaded_image = files.upload()

# Process the uploaded image
for fn in uploaded_image.keys():
    img = Image.open(fn)
    img = img.convert('L')  # Convert to grayscale
    img = np.array(img)
    # Resize to match the model's expected input shape (64x64)
    img_resized = cv2.resize(img, (64, 64))  # Resize to 64x64 pixels
    img_normalized = img_resized / 255.0  # Normalize pixel values
    img_normalized = img_normalized.reshape(1, 64, 64, 1)  # Reshape for model input
    print(f"Processed Image Shape: {img_normalized.shape}")

    # Predict the emotion
    emotion_classes = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
    emotion_to_mood = {
        'Happy': 'uplifting',
        'Sad': 'calm',
        'Angry': 'relaxing',
        'Surprise': 'energetic',
        'Neutral': 'chill',
        'Fear': 'soothing',
        'Disgust': 'motivational'
    }

    predicted_emotion = model.predict(img_normalized)
    emotion_index = np.argmax(predicted_emotion)
    detected_emotion = emotion_classes[emotion_index]

    print(f"Detected Emotion: {detected_emotion}")

    # Map to music mood
    music_mood = emotion_to_mood.get(detected_emotion, 'chill')  # Default to 'chill' if emotion not found
    print(f"Music Mood: {music_mood}")

    # Set up Spotify API (replace with your credentials)
    SPOTIPY_CLIENT_ID = ''
    SPOTIPY_CLIENT_SECRET = ''
    SPOTIPY_REDIRECT_URI = ''  # Must match Spotify app settings

    oauth = SpotifyOAuth(client_id=SPOTIPY_CLIENT_ID,
                                                   client_secret=SPOTIPY_CLIENT_SECRET,
                                                   redirect_uri=SPOTIPY_REDIRECT_URI,
                                                   scope="user-library-read")


# Get the authorization URL
    auth_url = oauth.get_authorize_url()

# Print the authorization URL and ask the user to authorize your application
    print(f"Please visit this URL to authorize your application: {auth_url}")

    import webbrowser
    webbrowser.open(auth_url)  # Open the authorization URL in a web browser
    print("After authorizing, paste the URL you were redirected to here:")
    redirect_url = input()  # Get the redirected URL from user input
    import urllib.parse as urlparse
    parsed_url = urlparse.urlparse(redirect_url)
    auth_code = urlparse.parse_qs(parsed_url.query)['code'][0]  # Extract the authorization code

    # Get the access token using the authorization code
    token_info = oauth.get_access_token(auth_code)


    # Create an instance of Spotify using the access token
    sp = spotipy.Spotify(auth=token_info['access_token'])

    # Search for mood-based playlist
    results = sp.search(q=music_mood, type='playlist', limit=1)

    if results['playlists']['items']:
        playlist = results['playlists']['items'][0]
        print(f"Recommended Playlist: {playlist['name']}")
        print(f"Playlist URL: {playlist['external_urls']['spotify']}")
    else:
        print("No playlist found for the detected mood.")


Saving WIN_20241015_14_49_51_Pro.jpg to WIN_20241015_14_49_51_Pro (2).jpg


Saving WIN_20241015_14_49_51_Pro.jpg to WIN_20241015_14_49_51_Pro (3).jpg
Processed Image Shape: (1, 64, 64, 1)
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 715ms/step
Detected Emotion: Angry
Music Mood: relaxing
Please visit this URL to authorize your application: https://accounts.spotify.com/authorize?client_id=7353e8b9f14d44019cdf20f0b2af9864&response_type=code&redirect_uri=https%3A%2F%2F4955-2409-40f4-1f-2ccb-8537-b558-c7d-75e7.ngrok-free.app%2Fcallback%2F&scope=user-library-read
After authorizing, paste the URL you were redirected to here:
https://4955-2409-40f4-1f-2ccb-8537-b558-c7d-75e7.ngrok-free.app/callback/?code=AQCcDKwLx4F7EvAbjVVq75cBcgNEBlaHRUZCaM20AF6A3byvi87FMwezy3vNUmNFNGjm1Lu3iqMzc9d3C9VmCkUcno4OCav9pkS6CR9bQiDhxBqI43fGPJ1gBst386DLRN5S47F1b11zzNHOH-LujxqMvpkRr61U2e26bV_NjbaJzt6ob8tv33a0mVi_NcuNxGyHsXGqMOcIQoMyg3wrBH5HCXXhWl35UQV9RkkbtAaOPo2z55mLwX5nVlLoxzAhNw
Recommended Playlist: Mind Relaxing Hindi Songs 🎧
Playlist URL: https://open.spotify.com/playlist/1dD2RAirtwoQQwno1mglZH


<ipython-input-4-dfd1a164997d>:83: DeprecationWarning: You're using 'as_dict = True'.get_access_token will return the token string directly in future versions. Please adjust your code accordingly, or use get_cached_token instead.
  token_info = oauth.get_access_token(auth_code)
